# Travail 3

> Je réutilise mon code du TD 1 pour la méthode Bayésienne que j'avais implémentée. 

In [95]:
import pandas as pd
import numpy as np
from math import prod
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [96]:
df_spambase = pd.read_csv('./csv_result-spambase_app_bool_sans_3_att.csv')
target = df_spambase.columns[-1]
df_spambase.drop(columns=['id'], axis=1, inplace=True)
df_spambase.head(3)

,word_freq_make_binarized,word_freq_address_binarized,word_freq_all_binarized,word_freq_3d_binarized,word_freq_our_binarized,word_freq_over_binarized,word_freq_remove_binarized,word_freq_internet_binarized,word_freq_order_binarized,word_freq_mail_binarized,...,word_freq_edu_binarized,word_freq_table_binarized,word_freq_conference_binarized,char_freq_;_binarized,char_freq_(_binarized,char_freq_[_binarized,char_freq_!_binarized,char_freq_$_binarized,char_freq_#_binarized,Class
0,1,1,1,0,1,1,1,0,1,1,...,1,0,0,0,1,0,1,1,1,1
1,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,1,1,0,1


> Tous les attribaux sont catégoriques (nominaux), mais je garde le même code.

In [97]:
list_categorical_columns = df_spambase.dtypes[df_spambase.dtypes != float].index.to_list()[:-1]
list_numerical_columns = df_spambase.dtypes[df_spambase.dtypes == float].index.to_list()
set_categorical, set_numerical = set(list_categorical_columns), set(list_numerical_columns)
print(f'Categorical columns: {list_categorical_columns}\nNumerical columns: {list_numerical_columns}')

Categorical columns: ['word_freq_make_binarized', 'word_freq_address_binarized', 'word_freq_all_binarized', 'word_freq_3d_binarized', 'word_freq_our_binarized', 'word_freq_over_binarized', 'word_freq_remove_binarized', 'word_freq_internet_binarized', 'word_freq_order_binarized', 'word_freq_mail_binarized', 'word_freq_receive_binarized', 'word_freq_will_binarized', 'word_freq_people_binarized', 'word_freq_report_binarized', 'word_freq_addresses_binarized', 'word_freq_free_binarized', 'word_freq_business_binarized', 'word_freq_email_binarized', 'word_freq_you_binarized', 'word_freq_credit_binarized', 'word_freq_your_binarized', 'word_freq_font_binarized', 'word_freq_000_binarized', 'word_freq_money_binarized', 'word_freq_hp_binarized', 'word_freq_hpl_binarized', 'word_freq_george_binarized', 'word_freq_650_binarized', 'word_freq_lab_binarized', 'word_freq_labs_binarized', 'word_freq_telnet_binarized', 'word_freq_857_binarized', 'word_freq_data_binarized', 'word_freq_415_binarized', 'word

> Calcul des valeurs possibles pour chaque colonne, ici c'est déjà binaire mais le code fonctionne dans le cas général non binaire.

In [98]:
list_values = {label: [x for x in np.unique(df_spambase[label].values) if x != '?'] for label in list_categorical_columns}
list_values

{'word_freq_make_binarized': [0, 1],
 'word_freq_address_binarized': [0, 1],
 'word_freq_all_binarized': [0, 1],
 'word_freq_3d_binarized': [0, 1],
 'word_freq_our_binarized': [0, 1],
 'word_freq_over_binarized': [0, 1],
 'word_freq_remove_binarized': [0, 1],
 'word_freq_internet_binarized': [0, 1],
 'word_freq_order_binarized': [0, 1],
 'word_freq_mail_binarized': [0, 1],
 'word_freq_receive_binarized': [0, 1],
 'word_freq_will_binarized': [0, 1],
 'word_freq_people_binarized': [0, 1],
 'word_freq_report_binarized': [0, 1],
 'word_freq_addresses_binarized': [0, 1],
 'word_freq_free_binarized': [0, 1],
 'word_freq_business_binarized': [0, 1],
 'word_freq_email_binarized': [0, 1],
 'word_freq_you_binarized': [0, 1],
 'word_freq_credit_binarized': [0, 1],
 'word_freq_your_binarized': [0, 1],
 'word_freq_font_binarized': [0, 1],
 'word_freq_000_binarized': [0, 1],
 'word_freq_money_binarized': [0, 1],
 'word_freq_hp_binarized': [0, 1],
 'word_freq_hpl_binarized': [0, 1],
 'word_freq_georg

> Initialisation du dictionnaire des fréquences.

In [99]:
dictionnary_frequencies = {
    column_name: {label: {'yes': 0, 'no':0} for label in list_values[column_name]} for column_name in list_categorical_columns
}
print(f'dictionnary_frequencies = {dictionnary_frequencies}')

dictionnary_frequencies = {'word_freq_make_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_address_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_all_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_3d_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_our_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_over_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_remove_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_internet_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_order_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_mail_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_receive_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_will_binarized': {0: {'yes': 0, 'no': 0}, 1: {'yes': 0, 'no': 0}}, 'word_freq_people_binarized'

> Remplissage du dictionnaire des fréquences. <br>
> Modification par rapport au TD1: la normalisation est effectuée dès la première boucle.

In [100]:
# dictionnary with number of occurences for Q4
dictionnary_occurences = {
    column_name: {label: {'yes': 0, 'no':0} for label in list_values[column_name]} for column_name in list_categorical_columns
}
epsilon = 1
for column_name in list_categorical_columns:
    total_entries = 0
    mode = len(list_values[column_name])

    # First loop to get the number of yes and no corresponding to the label
    for label in list_values[column_name]:
        n_yes = len(df_spambase[(df_spambase[column_name]==label) & (df_spambase['Class']==1)])
        n_no = len(df_spambase[(df_spambase[column_name]==label) & (df_spambase['Class']==0)])
        total_entries = n_yes + n_no

        dictionnary_occurences[column_name][label]['yes'] = n_yes
        dictionnary_occurences[column_name][label]['no'] = n_no

        dictionnary_frequencies[column_name][label]['yes'] = (n_yes + epsilon)/(total_entries+mode*epsilon)
        dictionnary_frequencies[column_name][label]['no'] = (n_no + epsilon)/(total_entries+mode*epsilon)

    print(f'After adding the number of occurences of {column_name}, dictonnary_frequencies = {dictionnary_frequencies[column_name]}')

After adding the number of occurences of word_freq_make_binarized, dictonnary_frequencies = {0: {'yes': 0.3273921200750469, 'no': 0.6726078799249531}, 1: {'yes': 0.6078224101479915, 'no': 0.39217758985200846}}
After adding the number of occurences of word_freq_address_binarized, dictonnary_frequencies = {0: {'yes': 0.31835879005690326, 'no': 0.6816412099430967}, 1: {'yes': 0.6944099378881987, 'no': 0.30559006211180123}}
After adding the number of occurences of word_freq_all_binarized, dictonnary_frequencies = {0: {'yes': 0.25676784249384743, 'no': 0.7432321575061526}, 1: {'yes': 0.5838218053927315, 'no': 0.41617819460726846}}
After adding the number of occurences of word_freq_3d_binarized, dictonnary_frequencies = {0: {'yes': 0.38727760175481357, 'no': 0.6127223982451865}, 1: {'yes': 0.8048780487804879, 'no': 0.1951219512195122}}
After adding the number of occurences of word_freq_our_binarized, dictonnary_frequencies = {0: {'yes': 0.2372093023255814, 'no': 0.7627906976744186}, 1: {'yes

#### Test du du modèle 
> Je n'ai pas implémenté l'apprentissage continu pour chaque nouvel instance du test set.

In [101]:
df_spambase_test = pd.read_csv('./csv_result-spambase_test_bool_sans_3_att.csv')
target = df_spambase_test.columns[-1]
df_spambase_test.drop(columns=['id'], axis=1, inplace=True)
df_spambase_test.head(3)

,word_freq_make_binarized,word_freq_address_binarized,word_freq_all_binarized,word_freq_3d_binarized,word_freq_our_binarized,word_freq_over_binarized,word_freq_remove_binarized,word_freq_internet_binarized,word_freq_order_binarized,word_freq_mail_binarized,...,word_freq_edu_binarized,word_freq_table_binarized,word_freq_conference_binarized,char_freq_;_binarized,char_freq_(_binarized,char_freq_[_binarized,char_freq_!_binarized,char_freq_$_binarized,char_freq_#_binarized,Class
0,0,1,0,0,1,0,1,1,0,1,...,0,0,0,0,1,1,1,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
dictionnary_frequencies

{'word_freq_make_binarized': {0: {'yes': 0.3273921200750469,
   'no': 0.6726078799249531},
  1: {'yes': 0.6078224101479915, 'no': 0.39217758985200846}},
 'word_freq_address_binarized': {0: {'yes': 0.31835879005690326,
   'no': 0.6816412099430967},
  1: {'yes': 0.6944099378881987, 'no': 0.30559006211180123}},
 'word_freq_all_binarized': {0: {'yes': 0.25676784249384743,
   'no': 0.7432321575061526},
  1: {'yes': 0.5838218053927315, 'no': 0.41617819460726846}},
 'word_freq_3d_binarized': {0: {'yes': 0.38727760175481357,
   'no': 0.6127223982451865},
  1: {'yes': 0.8048780487804879, 'no': 0.1951219512195122}},
 'word_freq_our_binarized': {0: {'yes': 0.2372093023255814,
   'no': 0.7627906976744186},
  1: {'yes': 0.6457800511508951, 'no': 0.35421994884910485}},
 'word_freq_over_binarized': {0: {'yes': 0.3115919629057187,
   'no': 0.6884080370942813},
  1: {'yes': 0.6754675467546755, 'no': 0.32453245324532454}},
 'word_freq_remove_binarized': {0: {'yes': 0.2732592159157402,
   'no': 0.7267407

#### Prédictions

In [103]:
df = dictionnary_frequencies

def predict_instance(i, df_test):
    """ Make the prediction of the i-th instance in the test_dataframe"""
    proba_yes = prod(df[c][df_test.iloc[i][c]]['yes'] for c in list_categorical_columns) 
    proba_no = prod(df[c][df_test.iloc[i][c]]['no'] for c in list_categorical_columns)
    proba_yes, proba_no = proba_yes/(proba_yes+proba_no), proba_no/(proba_yes+proba_no)
    return 1 if proba_yes > proba_no else 0

def predict(df_test):
    """ Make the prediction on the whole test_dataframe"""
    return [predict_instance(i, df_test) for i in range(len(df_test))]

list_predictions = predict(df_test = df_spambase_test)
cm = confusion_matrix(df_spambase_test['Class'].to_list(), list_predictions, labels=[1,0])
print(f'Confusion matrix: \n{cm}')
print(classification_report(df_spambase_test['Class'].to_list(), list_predictions, labels=[1,0]))

Confusion matrix: 
[[ 28 165]
 [  0 268]]
              precision    recall  f1-score   support

           1       1.00      0.15      0.25       193
           0       0.62      1.00      0.76       268

    accuracy                           0.64       461
   macro avg       0.81      0.57      0.51       461
weighted avg       0.78      0.64      0.55       461



#### Q1 - Analyse des résultats
> Le modèle obtenu est très précis (100%) mais il y a beaucoup de faux négatifs. <br>
> L'accuracy est seulement de 64%. <br>
> Il faudrait probablement implémenter la mise à jour des probabilités au fur et à mesure du test pour améliorer les résultats. <br>
> A noter que les métriques sont obtenues avec le test set et non par validation croisée, ce qui pourrait être à l'origine de moins bons résultats. <br>

#### Q2 - Origine du bon comportement avec Weka
> Deux hypothèses pour un meilleur comportement avec Weka:
>> Métriques calculées par validation croisées, et train set plus ou moins redondant, dont les plis de test de la validation croisée aurait des instances plus ou moins déjà vues. <br>
>> Mise à jour des probabilités au fil des tests, et donc finalement de meilleurs résultats.

#### Q3 - Hypothèses
> Si les attributs ne sont pas indépendants, on ne peut pas calculer les probabilités conditionnelles simplement. Il est probable que certains attributs ne soient pas indépendants, car certains mots doivent probablement se trouver dans les mêmes types de mail. Par exemple, 'you' et 'your' ne sont probablement pas indépendants.<br>
> De même, si ces données ne représentent pas ce qu'on touve dans les mails, les probabilités seront erronnées. <br>

#### Q4 - Probabilités a-priori
> Elles sont contenues dans le dictionnaire des fréquences.

In [104]:
df

{'word_freq_make_binarized': {0: {'yes': 0.3273921200750469,
   'no': 0.6726078799249531},
  1: {'yes': 0.6078224101479915, 'no': 0.39217758985200846}},
 'word_freq_address_binarized': {0: {'yes': 0.31835879005690326,
   'no': 0.6816412099430967},
  1: {'yes': 0.6944099378881987, 'no': 0.30559006211180123}},
 'word_freq_all_binarized': {0: {'yes': 0.25676784249384743,
   'no': 0.7432321575061526},
  1: {'yes': 0.5838218053927315, 'no': 0.41617819460726846}},
 'word_freq_3d_binarized': {0: {'yes': 0.38727760175481357,
   'no': 0.6127223982451865},
  1: {'yes': 0.8048780487804879, 'no': 0.1951219512195122}},
 'word_freq_our_binarized': {0: {'yes': 0.2372093023255814,
   'no': 0.7627906976744186},
  1: {'yes': 0.6457800511508951, 'no': 0.35421994884910485}},
 'word_freq_over_binarized': {0: {'yes': 0.3115919629057187,
   'no': 0.6884080370942813},
  1: {'yes': 0.6754675467546755, 'no': 0.32453245324532454}},
 'word_freq_remove_binarized': {0: {'yes': 0.2732592159157402,
   'no': 0.7267407

In [105]:
dictionnary_occurences

{'word_freq_make_binarized': {0: {'yes': 1046, 'no': 2150},
  1: {'yes': 574, 'no': 370}},
 'word_freq_address_binarized': {0: {'yes': 1062, 'no': 2275},
  1: {'yes': 558, 'no': 245}},
 'word_freq_all_binarized': {0: {'yes': 625, 'no': 1811},
  1: {'yes': 995, 'no': 709}},
 'word_freq_3d_binarized': {0: {'yes': 1588, 'no': 2513},
  1: {'yes': 32, 'no': 7}},
 'word_freq_our_binarized': {0: {'yes': 611, 'no': 1967},
  1: {'yes': 1009, 'no': 553}},
 'word_freq_over_binarized': {0: {'yes': 1007, 'no': 2226},
  1: {'yes': 613, 'no': 294}},
 'word_freq_remove_binarized': {0: {'yes': 933, 'no': 2483},
  1: {'yes': 687, 'no': 37}},
 'word_freq_internet_binarized': {0: {'yes': 1069, 'no': 2334},
  1: {'yes': 551, 'no': 186}},
 'word_freq_order_binarized': {0: {'yes': 1119, 'no': 2321},
  1: {'yes': 501, 'no': 199}},
 'word_freq_mail_binarized': {0: {'yes': 885, 'no': 2093},
  1: {'yes': 735, 'no': 427}},
 'word_freq_receive_binarized': {0: {'yes': 1110, 'no': 2388},
  1: {'yes': 510, 'no': 132}

In [106]:
dictionnary_occurences['word_freq_3d_binarized']

{0: {'yes': 1588, 'no': 2513}, 1: {'yes': 32, 'no': 7}}

> Le nombre d'occurences n'est pas exactement le même que dans le sujet. <br>
La probabilité conditionnelle se calcule comme suit:
$
\begin{equation}
P(3d|spam)=\frac{P(Spam|3d)P(3d)}{P(spam)}
\end{equation}
$ <br>

D'après le dictionnaire des occurences, $P(Spam|3d) = \frac{32}{39}$, $P(no\_Spam|3d) = \frac{7}{39}$

> On récupère ensuite P(spam) et P(3d)

In [107]:
P_spam = len(df_spambase[df_spambase['Class'] == 1]) / len(df_spambase)
P_no_spam = len(df_spambase[df_spambase['Class'] == 0]) / len(df_spambase)
print(f'P(spam) = {P_spam}\nP(no_spam) = {P_no_spam}')

P_3d = ((t:=dictionnary_occurences['word_freq_3d_binarized'][1])['yes'] + t['no']) / len(df_spambase)
print(f'P(3d) = {P_3d}')

P_3d_spam = ((32/39)*(P_3d))/P_spam
P_3d_no_spam = ((7/39)*(P_3d))/P_no_spam
print(f'P(3d|spam) = {P_3d_spam}\nP(3d|no_spam) = {P_3d_no_spam}')

P(spam) = 0.391304347826087
P(no_spam) = 0.6086956521739131
P(3d) = 0.009420289855072464
P(3d|spam) = 0.019753086419753086
P(3d|no_spam) = 0.002777777777777778


> Le mot 3d est très peu présent dans la base de données, et la probabilité qu'il soit dans le mail sachant que le mail est un spam est très faible. <br>
> La probabilité d'avoir 3d en sachant qu'on a eu de spam est encore plus faible.

#### Q6
> J'ai déjà testé le classifier sur le test set (qui contient de nombreuses instances, pas juste une). <br>
> En ajoutant des mots non spams, la prédiction peut changer. Elle peut passer de positive à négative. <br>